# Take raw (Carriage return removed from bash script) csv, and assign each row a random number for shuffling, and put in sqlite database

In [ ]:
# Put id, type, column in temp database
import sqlite3
import csv
import random
import sys

csv.field_size_limit(sys.maxsize)

random.seed(42)
n = 0

path = 'data/'
input_file = path + 'cr_removed.csv'

db_file = path + 'temp_database.db'

conn = sqlite3.connect(db_file)
c = conn.cursor()
c.execute("CREATE TABLE data (id TEXT, type TEXT, content TEXT, title TEXT, rnd INT);")

with open(input_file, "r") as f_in:
    reader = csv.reader(f_in)
    header = next(reader)  # Read the header

    for row in reader:
        id_value = row[1].replace("\n", " ")
        type_value = row[3].replace("\n", " ")
        content_value = row[5].replace("\n", " ")
        title_value = row[9].replace("\n", " ")
        rnd_value = int(
            random.uniform(-9223372036854775808, 9223372036854775807))  # sqlite max/min int values

        c.execute(
            "INSERT INTO data (id, type, content, title, rnd) VALUES (?, ?, ?, ?, ?)",
            (id_value, type_value, content_value, title_value, rnd_value),
        )
        n += 1
        if n % 1000 == 0 and n:
            print("inserted", n, "rows\r", end="")
conn.commit()
print()
print("finished")

# Shuffle and deduplicate database

In [ ]:
import sqlite3
import csv
import random
import sys

csv.field_size_limit(sys.maxsize)
conn = sqlite3.connect(db_file)
c = conn.cursor()

c.execute(
    """CREATE TABLE nodup AS
             SELECT id, type, content, title, rnd
             FROM data
             GROUP BY content
             ORDER BY rnd;"""
)

conn.commit()
conn.close()

# Delete the old database table to save space

In [ ]:
conn = sqlite3.connect(db_file)
c = conn.cursor()

c.execute("""DROP TABLE IF EXISTS data""")

# Commit the changes and close the connection
conn.commit()
conn.close()

# load shuffled and deduped data back from database into csv

In [ ]:
import csv
import sqlite3

# Set the batch size for reading the data from the SQLite table
batch_size = 400000

output_file = path + "shuffled_dataset.csv"
conn = sqlite3.connect(db_file)
c = conn.cursor()

# Execute a SELECT statement to get the column names
c.execute("SELECT id, type, content, title FROM nodup LIMIT 0")
column_names = [description[0] for description in c.description]
print("column names:", column_names)

# Open the output file and write only the header row
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(column_names)

    # Execute a SELECT statement to read the data from the table in batches
    offset = 0
    while True:
        c.execute(
            f"SELECT id, type, content, title FROM nodup LIMIT {batch_size} OFFSET {offset}"
        )
        rows = c.fetchall()

        # Break the loop if no more rows are returned
        if not rows:
            break

        # Write the rows to the output file
        writer.writerows(rows)

        # Increment the offset to read the next batch of rows
        offset += batch_size
# Close the database connection
conn.close()

# Delete entire database, to save space

In [ ]:
import os
os.remove(db_file)

# Parquetise csv in chunks of 50000 rows (approx 100M)

In [ ]:
## Parquetise sql filtered data
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


csv_file_path = output_file
parquet_file_prefix = path + "shuffled_deduped.parquet/file_"
parquet_file_suffix = ".parquet"

column_dtypes = {"id": int, "type": object, "content": object, "title": object}
# desired_columns = ['id', 'content', 'type']
chunk_size = 50000  # number of rows per chunk

# Iterate over the CSV file in chunks and write each chunk to a separate Parquet file
for i, chunk in enumerate(
    pd.read_csv(csv_file_path, chunksize=chunk_size, dtype=column_dtypes)
):
    # Filter rows based on the "type" column
    # chunk = chunk.query('type == "fake" or type == "reliable"').copy()

    # Create the filename for the current chunk
    ident = str(i).zfill(4)
    parquet_file_path = parquet_file_prefix + ident + parquet_file_suffix

    # Write the current chunk to a Parquet file
    table = pa.Table.from_pandas(chunk)
    pq.write_table(table, parquet_file_path)
    print("chunk", ident,'\r', end='')
print()

In [ ]:
import pandas as pd
import random

random.seed(42)

file_name = path + "shuffled_deduped.parquet"
df = pd.read_parquet(file_name, columns=["type", "content"])
# Filter out the 'empty' and 'unknown' types
df = df[df["type"].isin(["", "unknown"]) == False]
df_size = df.shape[0]

n_non_reliable = 3897597
n_reliable = 1808242
downsample_factor = n_reliable / n_non_reliable


train_size = round(df_size * 0.8)
val_size = (df_size - train_size) // 2
test_size = df_size - train_size - val_size


def dicethrow(thistype):
    if thistype == "reliable":
        return True
    rand = random.random()
    if rand > downsample_factor:
        return False
    return True


# Generate y values for models.
df["class"] = df["type"] == "reliable"

train_data = df.iloc[:train_size]
val_data = df.iloc[train_size : train_size + val_size]
test_data = df.iloc[train_size + val_size :]

train_data = train_data[train_data["type"].map(dicethrow)]


# WRITE PARQUETS
def parquetise(df, path: str, set_type: str, chunk_size=100000):
    """Parquetise some dataframe
    input: df (dataframe),
    path: The relative path, such as 'many_parquet_files_here.parquet/'
    set_type, string. typically "train", "val" or "test",
    chunk_size (optional): the number of rows per chunk"""

    file_n = 0
    df_length = len(df)
    for i in range(0, df_length, chunk_size):
        ident = str(file_n).zfill(6)

        chunk_df = df.iloc[i : min(i + chunk_size, df_length)]
        filename = f"{set_type}_{ident}.parquet"
        print("Writing:", filename, "\r", end="")
        chunk_df.to_parquet(path + filename, index=False)
        file_n += 1
    print()
    print("done!")


parquetise(train_data, path+"train.parquet/", "train")
parquetise(val_data, path+"val.parquet/", "val")
parquetise(test_data, path+"test.parquet/", "test")

